# Introduction

This notebook demonstrates the Wise Platform **Quote - Recipient - Transfer - Found** flow via Wise Platofrm API. 

To begin obtain your API key for the Wise Sandbox environment at [https://sandbox.transferwise.tech/home](https://sandbox.transferwise.tech/home). 
Navigate to **Your account** -> **Integrations and tools** -> **API tokens** to generate or retrieve your API key.

Ensure that the API key is stored securely and is not exposed in public repositories or shared with unauthorized individuals.

In [1]:
# Wise API Key
api_key = "your-wise-api-key"

# Wise Client

Wise Client is a utility designed to streamline the process of sending and receiving API messages to and from the Wise Platform API. 

In [13]:
import pandas as pd
pd.set_option('display.max_rows', None)

import requests

# Wise API Client
class WiseAPI:
  BASE_URL = "https://api.sandbox.transferwise.tech"
  def __init__(self, api_key):
    self.api_key = api_key
    self.headers = {"Authorization": f"Bearer {self.api_key}"}

  def get(self, endpoint):
    url = f"{self.BASE_URL}/{endpoint}"
    response = requests.get(url, headers=self.headers)
    response.raise_for_status()
    return response.json()

  def post(self, endpoint, payload):
    url = f"{self.BASE_URL}/{endpoint}"
    response = requests.post(url, json=payload, headers=self.headers)
    response.raise_for_status()
    return response.json()

wise= WiseAPI(api_key)

# Step 1: Get user profile

Profiles are connected to a User account and are either personal or business.
> https://docs.wise.com/api-docs/api-reference/profile

In [3]:
profiles = wise.get(endpoint = "/v2/profiles")
profiles_df = pd.json_normalize(profiles)
profiles_df.T

,0
type,PERSONAL
id,28662219
publicId,71c708be-112d-4362-be28-72c8f5015b0d
userId,13023867
email,1740694267818@test.dev
createdAt,2025-02-27T22:13:04
updatedAt,2025-04-27T18:22:33
obfuscated,False
currentState,VISIBLE
firstName,Michal


# Step 2: Create a quote

The quote resource defines the basic information required for a Wise transfer - the currencies to send between, the amount to send and the profile who is sending the money. The profile must be included when creating a quote.

> https://docs.wise.com/api-docs/api-reference/quote

In [4]:
profile_id = profiles_df['id'][0]
quote = wise.post(endpoint = f"/v3/profiles/{profile_id}/quotes", payload = {
      "sourceCurrency": "USD",
      "targetCurrency": "GBP",
      "sourceAmount": 100,
      "targetAmount": None,
      "targetAccount": None,
      "preferredPayIn": "BALANCE"
    })
quote_df = pd.json_normalize(quote)
quote_df.T

,0
sourceAmount,100.0
guaranteedTargetAmountAllowed,False
targetAmountAllowed,True
preferredPayIn,BALANCE
paymentOptions,"[{'formattedEstimatedDelivery': 'in seconds', ..."
notices,[]
rateTimestamp,2025-04-27T18:35:42Z
clientId,transferwise-personal-tokens
expirationTime,2025-04-27T19:07:10Z
id,9394b252-0ed4-404f-b075-3b7b51c3bf00


### List Payment Options

In [5]:
df = pd.json_normalize(quote_df['paymentOptions'][0])
df

,formattedEstimatedDelivery,estimatedDeliveryDelays,allowedProfileTypes,feePercentage,estimatedDelivery,disabled,sourceAmount,targetAmount,sourceCurrency,targetCurrency,...,price.priceDecisionReferenceId,fee.transferwise,fee.payIn,fee.discount,fee.total,fee.priceSetId,fee.partner,disabledReason.code,disabledReason.message,disabledReason.arguments
0,in seconds,[],"[PERSONAL, BUSINESS]",0.0150,2025-04-27T18:37:10Z,False,100.0,74.01,USD,GBP,...,59e227e2-d0dd-4c6a-7dde-18ab100230ff,1.25,0.25,0,1.50,384,0.0,NaN,NaN,NaN
1,by Monday,[],"[PERSONAL, BUSINESS]",0.0734,2025-04-28T14:20:00Z,False,100.0,69.62,USD,GBP,...,59e227e2-d0dd-4c6a-7dde-18ab100230ff,1.23,6.11,0,7.34,384,0.0,NaN,NaN,NaN
2,in seconds,[],"[PERSONAL, BUSINESS]",0.0253,2025-04-27T18:37:10Z,False,100.0,73.24,USD,GBP,...,59e227e2-d0dd-4c6a-7dde-18ab100230ff,1.24,1.29,0,2.53,384,0.0,NaN,NaN,NaN
3,in seconds,[],"[PERSONAL, BUSINESS]",0.0739,2025-04-27T18:37:10Z,False,100.0,69.59,USD,GBP,...,59e227e2-d0dd-4c6a-7dde-18ab100230ff,1.23,6.16,0,7.39,384,0.0,NaN,NaN,NaN
4,"by Monday, May 5",[],"[PERSONAL, BUSINESS]",0.0734,2025-05-05T18:00:00Z,False,100.0,69.62,USD,GBP,...,59e227e2-d0dd-4c6a-7dde-18ab100230ff,1.23,6.11,0,7.34,384,0.0,NaN,NaN,NaN
5,in seconds,[],"[PERSONAL, BUSINESS]",0.0611,2025-04-27T18:37:10Z,False,100.0,70.55,USD,GBP,...,59e227e2-d0dd-4c6a-7dde-18ab100230ff,1.23,4.88,0,6.11,384,0.0,NaN,NaN,NaN
6,in seconds,[],"[PERSONAL, BUSINESS]",0.0743,2025-04-27T18:37:10Z,False,100.0,69.55,USD,GBP,...,59e227e2-d0dd-4c6a-7dde-18ab100230ff,1.23,6.20,0,7.43,384,0.0,NaN,NaN,NaN
7,in seconds,[],"[PERSONAL, BUSINESS]",0.0253,2025-04-27T18:37:10Z,False,100.0,73.24,USD,GBP,...,59e227e2-d0dd-4c6a-7dde-18ab100230ff,1.24,1.29,0,2.53,384,0.0,NaN,NaN,NaN
8,in seconds,[],"[PERSONAL, BUSINESS]",0.0253,2025-04-27T18:37:10Z,False,100.0,73.24,USD,GBP,...,59e227e2-d0dd-4c6a-7dde-18ab100230ff,1.24,1.29,0,2.53,384,0.0,NaN,NaN,NaN
9,in seconds,[],"[PERSONAL, BUSINESS]",0.0253,2025-04-27T18:37:10Z,False,100.0,73.24,USD,GBP,...,59e227e2-d0dd-4c6a-7dde-18ab100230ff,1.24,1.29,0,2.53,384,0.0,NaN,NaN,NaN


# Step 3: Create a Recipient Account

Recipient or beneficiary is the one who will receive the funds.

> https://docs.wise.com/api-docs/api-reference/recipient

In [6]:
recipient = wise.post(endpoint = f"/v1/accounts", payload = {
          "currency": "GBP",
          "type": "sort_code",
          "profile": str(profile_id),
          "ownedByCustomer": True,
          "accountHolderName": "John Doe",
           "details": {
              "legalType": "PRIVATE",
              "sortCode": "040075",
              "accountNumber": "37778842",
              "dateOfBirth": "1961-01-01"
           }
         })

recipient_df = pd.json_normalize(recipient)
recipient_df.T

,0
id,701042065
business,None
profile,28662219
accountHolderName,John Doe
currency,GBP
country,GB
type,sort_code
user,13023867
active,True
ownedByCustomer,True


# Step 4: Create a transfer

A transfer is a payment order to recipient account based on a quote.
> https://docs.wise.com/api-docs/api-reference/transfer

In [7]:
import uuid

transfer = wise.post(endpoint = f"/v1/transfers", payload = {
      "targetAccount": recipient['id'],
      "quoteUuid": quote['id'],
      "customerTransactionId": str(uuid.uuid4()),
    })

transfer_df = pd.json_normalize(transfer)
transfer_df.T

,0
id,55027732
user,13023867
targetAccount,701042065
sourceAccount,None
quote,None
quoteUuid,9394b252-0ed4-404f-b075-3b7b51c3bf00
status,incoming_payment_waiting
reference,
rate,0.751377
created,2025-04-27 18:37:24


# Step 5: Fund the transfer

This is the final step for executing payouts when using a balance with Wise. Upon calling the endpoint, Wise will begin the processing of the transfer, depending on the status of funds.

> https://docs.wise.com/api-docs/api-reference/transfer#fund

In [ ]:
funding = wise.post(endpoint = f"/v3/profiles/{profile_id}/transfers/{transfer['id']}/payments", payload = {   
    "type":"BALANCE"
  })
funding_df = pd.json_normalize(funding)
funding_df.T
